In [5]:
import os
from bs4 import BeautifulSoup

def find_all_html_files(folder_path):
    html_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.html') or file.endswith('.htm'):
                full_path = os.path.join(root, file)
                html_files.append(full_path)
    return html_files

def extract_meta_tags_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    canonical = None
    desc = None
    keyword = None

    link_tag = soup.find('link', rel='canonical')
    if link_tag and link_tag.has_attr('href'):
        canonical = link_tag['href']

    if not canonical:
        og_url_tag = soup.find('meta', property='og:url')
        if og_url_tag and og_url_tag.has_attr('content'):
            canonical = og_url_tag['content']

    desc_tag = soup.find('meta', attrs={'name': 'description'})
    if desc_tag and desc_tag.has_attr('content'):
        desc = desc_tag['content']

    keyword_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keyword_tag and keyword_tag.has_attr('content'):
        keyword = keyword_tag['content']

    return canonical, desc, keyword

def format_hashtags_multilang(keywords):
    if not keywords:
        return '없음'
    # 아랍어 쉼표(،)와 영어 쉼표(,) 모두 통일하여 ','로 대체
    for sep in ['،', ',']:
        keywords = keywords.replace(sep, ',')
    tags = []
    for kw in keywords.split(','):
        kw = kw.strip()
        if kw:
            # 단어 내부 공백 제거
            kw_no_space = kw.replace(' ', '')
            if not kw_no_space.startswith('#'):
                kw_no_space = '#' + kw_no_space
            tags.append(kw_no_space)
    return ' '.join(tags) if tags else '없음'

def create_x_post_format(folder_path):
    html_files = find_all_html_files(folder_path)
    posts = []

    for file_path in html_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            html_content = f.read()
            canonical, desc, keyword = extract_meta_tags_from_html(html_content)
            canonical = canonical if canonical else '없음'
#             desc = desc if desc else '없음'
#             hashtags = format_hashtags_multilang(keyword)
#             post = f"파일 경로: {file_path}\n{canonical}\n{desc}\n{hashtags}\n"
            post = f"{canonical}\n"
            posts.append(post)

    return posts

if __name__ == '__main__':
    folder_path = input("HTML 파일들이 있는 폴더 경로를 입력하세요: ").strip()
    posts = create_x_post_format(folder_path)
    if not posts:
        print("폴더 내에 HTML 파일이 없거나 메타태그가 없습니다.")
    else:
        with open('result.txt', 'w', encoding='utf-8') as f:
            f.write('\n'.join(posts))
        print("결과가 result.txt 파일로 저장되었습니다.")


HTML 파일들이 있는 폴더 경로를 입력하세요: C:\Users\USER\Documents\GitHub\emfls.github.io
결과가 result.txt 파일로 저장되었습니다.
